## This notebook queries various DAACs for sensor cadences of [Sentinel-1, Sentinel-2, and Landsat8] and saves them to a file
Inputs are area of interest (Polygon), date range (list of datetimes), and sampling resolution. Default sampling resolution is 1 degree longitude, 0.25 degree latitude.

In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:98% !important; }</style>")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# GIS imports
from shapely.geometry import Polygon, shape, Point
from opera_coverage import *
import geopandas as gpd
import pandas as pd
from shapely.geometry import box, Point
from rasterio.crs import CRS

# Misc imports
import numpy as np
from itertools import product
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import concurrent.futures
from typing import List

In [4]:
output_path = Path('../output_dfs')
output_path.mkdir(exist_ok=True)

In [5]:
# helper functions
import sys
import time

# wrapper function for get_coverage, returns string if thread crashed while running
def get_cadence_df(geo:Polygon, daterange:List[datetime]):
#     results = pd.DataFrame(columns=['center_x','center_y','startTime','geometry','sensor','fileID'])
    for i in range(3):
        try:
            df = get_coverage(['sentinel1','sentinel2','landsat8'], geo, daterange)
            df['center_x'] = geo.centroid.x
            df['center_y'] = geo.centroid.y
            return df
        except:
            print("Oops!", str(sys.exc_info()[0]), " occurred.")
            print(sys.exc_info()[1])
            print(sys.exc_info()[2])
            print("Attempt " + str(i) + " failed")
            time.sleep(2)
    return 'Thread crashed'

For continent geometries use Natural Earth Lowres from Geopandas

In [6]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [7]:
# creates Polygon or Multipolygon of each continent
def create_polygon(continent):
    df = world[world.continent == continent]
    return df.geometry.unary_union

In [8]:
polygon_dict = {key:value for key,value in zip(sorted(world.continent.unique()), list(map(create_polygon, sorted(world.continent.unique()))))}
del polygon_dict['Antarctica'],polygon_dict['Seven seas (open ocean)']

In [9]:
# main function, takes an area of interest as a polygon and a daterange as a list of two datetimes
# grid resolution can be changed from default values
def get_area_coverage(aoi: Polygon, date: List[datetime], x_res = 1, y_res = 0.25, radius = 0.1):

    df_points = grid_intersect(aoi, x_res, y_res, radius)
    results = lookup(df_points, date)
    
    return results

In [10]:
# creates an array of polygon circles within the rectangular bounds of the aoi, then takes intersection with the aoi, returns dataframe
def grid_intersect(aoi: Polygon, x_res, y_res, radius):
    
    x_start, y_start, x_end, y_end = aoi.bounds
    df = make_grid(x_start, y_start, x_end, y_end, x_res, y_res, radius)
    indices = df.geometry.intersects(aoi)
    
    return df[indices].reset_index(drop=True)

In [11]:
# for each geometry in dataframe, runs search through asf and hls search, reformats to dataframe
def lookup(df_points: gpd.GeoDataFrame, daterange) -> gpd.GeoDataFrame:
    results = []

    for i in range(len(df_points) // 200 + 1):
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            results += list(executor.map(get_cadence_df, tqdm(df_points.geometry[(i * 200):(i + 1) * 200]), [daterange] * 200))
    
    fail_geometries = [df_points.geometry[j] for j,data in enumerate(results) if type(data) is str]
    print(len(fail_geometries))
    
    for i in range(len(fail_geometries) // 200 + 1):
            with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
                results += list(executor.map(get_cadence_df, tqdm(fail_geometries[(i * 200):(i + 1) * 200]), [daterange] * 200))
    
    results = pd.concat([i for i in results if type(i) is not str])
        
    return results

In [12]:
# Make grid with specified x and y resolution, returns as geodataframe
def make_grid(x_start, y_start, x_end, y_end, x_res = 1, y_res = 0.25, radius = 0.1):
    lon_lat = list(product(
                np.linspace(x_start + (x_res/2), x_end - (x_res/2), int((x_end - x_start) / x_res + 1)), 
                np.linspace(y_start + (y_res/2), y_end - (y_res/2), int((y_end - y_start) / y_res + 1))
                ))

    geometry = [Point(lon, lat).buffer(radius) for lon, lat in lon_lat]

    center_x = [lon for lon, _ in lon_lat]
    center_y = [lat for _, lat in lon_lat]

    df = gpd.GeoDataFrame({'center_x':center_x, 'center_y':center_y}, geometry = geometry, crs=CRS.from_epsg(4326))
    
    return df

In [13]:
for key,value in polygon_dict.items():
    df = get_area_coverage(value, [datetime(2021,1,1),datetime(2021,2,1)], x_res = 1, y_res = 1, radius = 0.1)
    df.to_pickle(output_path/f"{key}2021_1_results.pkl")

100%|████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1690.23it/s]


Oops! <class 'pystac_client.exceptions.APIError'>  occurred.
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Attempt 0 failed
Oops! <class 'pystac_client.exceptions.APIError'>  occurred.
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Attempt 0 failed


100%|███████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 11367.60it/s]


Oops! <class 'pystac_client.exceptions.APIError'>  occurred.

Attempt 0 failed
Oops! <class 'pystac_client.exceptions.APIError'>  occurred.

Attempt 0 failed
Oops! <class 'pystac_client.exceptions.APIError'>  occurred.
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Attempt 0 failed


100%|███████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 12861.03it/s]


...

100%|████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 5826.84it/s]


Oops! <class 'pystac_client.exceptions.APIError'>  occurred.
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Attempt 0 failed


100%|████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 5264.06it/s]


Oops! <class 'pystac_client.exceptions.APIError'>  occurred.
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Attempt 0 failed
Oops! <class 'pystac_client.exceptions.APIError'>  occurred.
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Attempt 0 failed
0


0it [00:00, ?it/s]
